![](../img/dl_banner.jpg)

# 用tensorflow low level API构建多层感知器
#### \[稀牛学院 x 网易云课程\]《深度学习工程师(实战)》课程资料 by [@寒小阳](https://blog.csdn.net/han_xiaoyang)

## 问题背景：MNIST 手写数字
#### \[稀牛学院 x 网易云课程\]《深度学习工程师(实战)》课程资料 by [@寒小阳](https://blog.csdn.net/han_xiaoyang)

这是一个非常经典的问题，我们的对象是手写数字图片，我们需要根据手写数字图片的输入，构建线性的分类器(softmax分类器)去区分图片是手写数字0-9中的哪一个。这个问题的数据集，每一张图片的表示是长宽为28的矩阵，我们有时候会把它展开成784维的向量。MNIST手写数据集长成下面这样。

![MNIST Dataset](http://neuralnetworksanddeeplearning.com/images/mnist_100_digits.png)

更多的信息可以参考: http://yann.lecun.com/exdb/mnist/

## 0.多层感知器
#### \[稀牛学院 x 网易云课程\]《深度学习工程师(实战)》课程资料 by [@寒小阳](https://blog.csdn.net/han_xiaoyang)
多层感知器是最直接形式的神经网络，由多层非线性映射组成，每一层包含多个神经元。第一层称之为输入层，最后一层称之有输出层，中间的层称之为隐层。MLP并没有规定隐层的数量，因此可以根据各自的需求选择合适的隐层层数。且对于输出层神经元的个数也没有限制。 MLP神经网络结构模型如下。
![](./graphs/mlp.png)

## 1.引入工具库

In [5]:
import tensorflow as tf
#config = tf.ConfigProto()
#config.gpu_options.per_process_gpu_memory_fraction = 0.3
#session = tf.Session(config=config, ...)

## 2.设定超参数

In [6]:
# 训练超参数
learning_rate = 0.001
training_epochs = 25 # 训练总轮数
batch_size = 128 # 一批数据大小
display_step = 1 # 信息展示的频度

# 网络超参数
n_hidden_1 = 256 # 第1个隐层
n_hidden_2 = 256 # 第2个隐层
n_input = 784 # MNIST 数据输入(28*28*1=784)
n_classes = 10 # MNIST 总共10个手写数字类别

## 3.准备数据，设定参数变量

In [9]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [7]:
# 占位符
X = tf.placeholder(tf.float32, [None, 784], name='X_placeholder') # mnist数据集维度为28*28=784
Y = tf.placeholder(tf.int32, [None, 10], name='Y_placeholder') # 0-9总共有10个不同的手写数字类别

# 设定模型权重
weights = {
    'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1]), name='W1'),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2]), name='W2'),
    'out': tf.Variable(tf.random_normal([n_hidden_2, n_classes]), name='W')
}
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1]), name='b1'),
    'b2': tf.Variable(tf.random_normal([n_hidden_2]), name='b2'),
    'out': tf.Variable(tf.random_normal([n_classes]), name='bias')
}

## 4.完成模型构建

In [1]:
# 构建多层感知器
def multilayer_perceptron(x, weights, biases):
    # 第1个隐层，使用relu激活函数
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'], name='fc_1')
    layer_1 = tf.nn.relu(layer_1, name='relu_1')
    # 第2个隐层，使用relu激活函数
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'], name='fc_2')
    layer_2 = tf.nn.relu(layer_2, name='relu_2')
    # 输出层
    out_layer = tf.add(tf.matmul(layer_2, weights['out']), biases['out'], name='fc_3')
    return out_layer

## 5.计算损失与优化器设定

In [8]:
# 计算损失函数
pred = multilayer_perceptron(X, weights, biases)
loss_all = tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=Y, name='cross_entropy_loss')
loss = tf.reduce_mean(loss_all, name='avg_loss')

# Adam优化器
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)

# 初始化计算图中的变量（用默认值去赋值）
init = tf.global_variables_initializer()

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



## 6.在session当中完成计算图计算(损失计算与优化、参数更新迭代)
#### \[稀牛学院 x 网易云课程\]《深度学习工程师(实战)》课程资料 by [@寒小阳](https://blog.csdn.net/han_xiaoyang)

In [9]:
with tf.Session() as sess:
    sess.run(init)
    writer = tf.summary.FileWriter('./graphs/MLP_DNN', sess.graph)

    # 训练
    for epoch in range(training_epochs):
        avg_loss = 0.
        total_batch = int(mnist.train.num_examples/batch_size)
        # 遍历所有的batches
        for i in range(total_batch):
            batch_x, batch_y = mnist.train.next_batch(batch_size)
            # 使用optimizer进行优化
            _, l = sess.run([optimizer, loss], feed_dict={X: batch_x, Y: batch_y})
            # 求平均的损失
            avg_loss += l / total_batch
        # 每一步都展示信息
        if epoch % display_step == 0:
            print("第%03d轮训练" % (epoch+1), "当前损失为", \
                "{:.9f}".format(avg_loss))
    print("训练完成！")

    # 在测试集上评估
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(Y, 1))
    # 计算准确率
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    print("测试集准确率:", accuracy.eval({X: mnist.test.images, Y: mnist.test.labels}))
    writer.close()

InternalError: failed initializing StreamExecutor for CUDA device ordinal 0: Internal: failed call to cuDevicePrimaryCtxRetain: CUDA_ERROR_OUT_OF_MEMORY: out of memory; total memory reported: 7981694976

### 版权归 © 稀牛学院 所有 保留所有权利
![](./graphs/xiniu_neteasy.png)